In [1]:
import gym
import tensorflow as tf
import load_policy
import tf_util
import pickle
import numpy as np
from sklearn import preprocessing
import sklearn
print(sklearn.__version__)

0.19.2


In [2]:
model = "Humanoid-v2"
logfile = "logfile"

In [3]:
loaded = pickle.load( open( "/home/yousof/courses/cs294_DeepRL/hw1/expert_data/"+model+".pkl", "rb" ) )
print(type(loaded) )
for key in loaded.keys():
    print(key)
print (type (loaded["actions"]))
print (loaded["actions"].shape)
print (loaded["observations"].shape)

<class 'dict'>
observations
actions
<class 'numpy.ndarray'>
(20000, 1, 17)
(20000, 376)


For example in Humanoid-v2 with 20 roll outs we have
20 sets of training data. Each ahave 17 actions and 376 observations.
The would means the input of the model would be 376 and the output would be 17.


In [4]:
env = gym.make(model)
print('observation space:', env.observation_space)
print('action space:', env.action_space)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
observation space: Box(376,)
action space: Box(17,)


In [5]:
spc = env.observation_space
print(type(spc.shape))
print(env.observation_space.shape[0])

<class 'tuple'>
376


In [6]:
num_rollout = 20


x_i = loaded["observations"]
y_i = np.squeeze(loaded["actions"],axis=1)
num_examples = x_i.shape[0]

num_ins = x_i.shape[1]
num_outs = y_i.shape[1]

print (x_i.shape, y_i.shape)

bat = num_examples / num_rollout
# print(bat)
bat = int(bat)

x = np.array([], dtype=np.float16).reshape(0,num_ins * 2 )
y = np.array([], dtype=np.float16).reshape(0,num_outs)


for idx, i in enumerate(range(0,num_examples,bat)):
    start = i
    end = i+bat
    xpreK = x_i[start:end-1,:]
    xK = x_i[start+1:end,:]
    x = np.concatenate( (x,np.concatenate((xK, xK-xpreK ) , axis = 1) ), axis = 0)
    y = np.concatenate( (y , y_i[start+1:end,:]), axis = 0  )


print (x.shape, y.shape)

num_examples = x.shape[0]


# normalize the values
def normalize(x,mean=None,std=None):
    if (mean is None):
        mean = np.mean(x,axis=0)
        std = np.std(x,axis=0)
        normalized = (x-mean)/std
        return normalized, mean, std
    else:
        return (x-mean)/std

def unnormalize(x, mean, std):
    unnorm = x * std + mean
    return unnorm

x_norm, x_mean, x_std = normalize(x)
y_norm, y_mean, y_std = normalize(y)

print (x_norm.shape, x_mean.shape, x_std.shape)
print (y_norm.shape, y_mean.shape, y_std.shape)

# remove the nans from the normalized vector
x_mask = (x_std!=0)
y_mask = (y_std!=0)
t = np.tile(x_mask,(x_norm.shape[0],1))
x_norm = x_norm[t].reshape([num_examples,-1])
t = np.tile(y_mask,(y_norm.shape[0],1))
y_norm = y_norm[t].reshape([num_examples,-1])
print (x_norm.shape, y_norm.shape)

y_un = unnormalize(y_norm,y_mean,y_std)
print("error" ,np.sum(np.abs(y_un - y)))

print (np.isnan(x_norm).any(),np.isnan(y_norm).any(),
       np.isinf(x_norm).any(),np.isinf(y_norm).any())

(20000, 376) (20000, 17)
(19980, 752) (19980, 17)
(19980, 752) (752,) (752,)
(19980, 17) (17,) (17,)
(19980, 611) (19980, 17)
error 0.0024314872
False False False False


/home/yousof/Software/programming/anaconda3/envs/RL/lib/python3.5/site-packages/ipykernel_launcher.py:40: RuntimeWarning: invalid value encountered in true_divide


In [6]:
# lets make a dense network to learn frome the training data as easy as possible

model_in_size = x_norm.shape[1]
model_out_size = y_norm.shape[1]

x_place = tf.placeholder(tf.float16,shape=(None, model_in_size),name="input")
y_place = tf.placeholder(tf.float16,shape=(None, model_out_size),name="label")

mean_init = 0
std_init = 0.001
regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)

dens = tf.layers.dense(x_place,512,
                       activation=tf.nn.tanh,
                       kernel_initializer=tf.truncated_normal_initializer(stddev=std_init),
                       name = "dense1",
                       kernel_regularizer=regularizer)
# dens = tf.layers.dense(dens,1024,
#                        activation=tf.nn.sigmoid,
#                        kernel_initializer=tf.truncated_normal_initializer(stddev=std_init),
#                        name = "dense2",
#                        kernel_regularizer=regularizer)
# dens = tf.layers.dense(dens,128,
#                        activation=tf.nn.tanh,
#                        kernel_initializer=tf.truncated_normal_initializer(stddev=std_init),
#                        name = "dense3",
#                        kernel_regularizer=regularizer)
# dens = tf.layers.dense(dens,128,
#                        activation=tf.nn.tanh,
#                        kernel_initializer=tf.truncated_normal_initializer(stddev=std_init),
#                        name = "dense4",
#                        kernel_regularizer=regularizer)
pred = tf.layers.dense(dens,model_out_size,
                       activation=tf.nn.tanh,
                       kernel_initializer=tf.truncated_normal_initializer(stddev=std_init),
                       name = "pred",
                       kernel_regularizer=regularizer)

loss = tf.losses.mean_squared_error(y_place , pred)
optmizer_op = tf.train.RMSPropOptimizer(learning_rate=0.001)
train_op = optmizer_op.minimize(loss)

saver = tf.train.Saver()

In [7]:
def get_batches(batch_size,x,y):
    x_size = x.shape[1]
    y_size = y.shape[1]
    total_number = x.shape[0]
    xy = np.concatenate((x,y),axis=1)
    np.random.shuffle(xy)
    for batch_i in range(0,total_number,batch_size):
        start = batch_i
        end = batch_i+ batch_size
        end = min (end, total_number)
        x_out = xy[start:end,0:x_size]
        y_out = xy[start:end,x_size:]
        #print(x_out.shape,y_out.shape)
        yield x_out,y_out 

In [8]:
num_epoch = 5000
batch_size = 1024*16

#merged = tf.summary.merge_all()
writer = tf.summary.FileWriter(logfile, tf.get_default_graph())

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(num_epoch):
#         i_batch = 0
#         for batch_x, batch_y in get_batches(batch_size,x_norm,y_norm):
#             i_batch += 1
#             out = sess.run([loss ,train_op], feed_dict = {x_place: batch_x , y_place: batch_y})
#             print("Epoch: {} batch: {} loss: {}"
#                   .format(epoch,i_batch,out[0]))
        out = sess.run([loss ,train_op], feed_dict = {x_place: x_norm , y_place: y_norm})
        print("Epoch: {} loss: {}"
              .format(epoch,out[0]))
#         print("===================================================")
    save_path = saver.save(sess, "models/"+model+".ckpt")

Epoch: 0 loss: 1.0000182390213013
Epoch: 1 loss: 1.0000125169754028
Epoch: 2 loss: 1.0000070333480835
Epoch: 3 loss: 1.0000005960464478
Epoch: 4 loss: 0.9999939203262329
Epoch: 5 loss: 0.9999867677688599
Epoch: 6 loss: 0.9999793767929077
Epoch: 7 loss: 0.9999712705612183
Epoch: 8 loss: 0.9999619126319885
Epoch: 9 loss: 0.9999527335166931
Epoch: 10 loss: 0.99994295835495
Epoch: 11 loss: 0.9999321103096008
Epoch: 12 loss: 0.9999207854270935
Epoch: 13 loss: 0.9999092817306519
Epoch: 14 loss: 0.999897301197052
Epoch: 15 loss: 0.9998840689659119
Epoch: 16 loss: 0.999869704246521
Epoch: 17 loss: 0.9998544454574585
Epoch: 18 loss: 0.9998378753662109
Epoch: 19 loss: 0.9998209476470947
Epoch: 20 loss: 0.9998023509979248
Epoch: 21 loss: 0.9997826814651489
Epoch: 22 loss: 0.9997619986534119
Epoch: 23 loss: 0.9997405409812927
Epoch: 24 loss: 0.9997171759605408
Epoch: 25 loss: 0.9996933341026306
Epoch: 26 loss: 0.9996675252914429
Epoch: 27 loss: 0.999640703201294
Epoch: 28 loss: 0.9996116757392883


Epoch: 231 loss: 0.21379463374614716
Epoch: 232 loss: 0.21483778953552246
Epoch: 233 loss: 0.21492186188697815
Epoch: 234 loss: 0.2177252173423767
Epoch: 235 loss: 0.21630237996578217
Epoch: 236 loss: 0.21544308960437775
Epoch: 237 loss: 0.21731409430503845
Epoch: 238 loss: 0.22060562670230865
Epoch: 239 loss: 0.22329853475093842
Epoch: 240 loss: 0.2159707099199295
Epoch: 241 loss: 0.21130232512950897
Epoch: 242 loss: 0.2082638293504715
Epoch: 243 loss: 0.21032075583934784
Epoch: 244 loss: 0.212090864777565
Epoch: 245 loss: 0.2118103802204132
Epoch: 246 loss: 0.20625340938568115
Epoch: 247 loss: 0.2055218368768692
Epoch: 248 loss: 0.20925484597682953
Epoch: 249 loss: 0.21705138683319092
Epoch: 250 loss: 0.21705178916454315
Epoch: 251 loss: 0.21537692844867706
Epoch: 252 loss: 0.2084980607032776
Epoch: 253 loss: 0.20924879610538483
Epoch: 254 loss: 0.21317949891090393
Epoch: 255 loss: 0.21253961324691772
Epoch: 256 loss: 0.21267648041248322
Epoch: 257 loss: 0.20786535739898682
Epoch: 25

Epoch: 457 loss: 0.18117386102676392
Epoch: 458 loss: 0.1833992600440979
Epoch: 459 loss: 0.18511690199375153
Epoch: 460 loss: 0.18581314384937286
Epoch: 461 loss: 0.18526989221572876
Epoch: 462 loss: 0.1841128021478653
Epoch: 463 loss: 0.18326491117477417
Epoch: 464 loss: 0.18311268091201782
Epoch: 465 loss: 0.18280576169490814
Epoch: 466 loss: 0.1818561851978302
Epoch: 467 loss: 0.18173114955425262
Epoch: 468 loss: 0.18202847242355347
Epoch: 469 loss: 0.18285763263702393
Epoch: 470 loss: 0.18431660532951355
Epoch: 471 loss: 0.1837463676929474
Epoch: 472 loss: 0.1846296638250351
Epoch: 473 loss: 0.18441623449325562
Epoch: 474 loss: 0.18551911413669586
Epoch: 475 loss: 0.18538542091846466
Epoch: 476 loss: 0.18380676209926605
Epoch: 477 loss: 0.18174496293067932
Epoch: 478 loss: 0.17969493567943573
Epoch: 479 loss: 0.17879152297973633
Epoch: 480 loss: 0.17890147864818573
Epoch: 481 loss: 0.17965520918369293
Epoch: 482 loss: 0.18121680617332458
Epoch: 483 loss: 0.18296769261360168
Epoch:

Epoch: 683 loss: 0.17264686524868011
Epoch: 684 loss: 0.17544420063495636
Epoch: 685 loss: 0.1763247400522232
Epoch: 686 loss: 0.17659053206443787
Epoch: 687 loss: 0.1746746450662613
Epoch: 688 loss: 0.17355163395404816
Epoch: 689 loss: 0.1721772998571396
Epoch: 690 loss: 0.17134320735931396
Epoch: 691 loss: 0.17087064683437347
Epoch: 692 loss: 0.1700633317232132
Epoch: 693 loss: 0.17014065384864807
Epoch: 694 loss: 0.16969065368175507
Epoch: 695 loss: 0.17035788297653198
Epoch: 696 loss: 0.17100705206394196
Epoch: 697 loss: 0.17164109647274017
Epoch: 698 loss: 0.17261724174022675
Epoch: 699 loss: 0.17082466185092926
Epoch: 700 loss: 0.16979384422302246
Epoch: 701 loss: 0.16851098835468292
Epoch: 702 loss: 0.16790728271007538
Epoch: 703 loss: 0.1684032678604126
Epoch: 704 loss: 0.16936416923999786
Epoch: 705 loss: 0.1719728708267212
Epoch: 706 loss: 0.1735696792602539
Epoch: 707 loss: 0.1739072948694229
Epoch: 708 loss: 0.17393700778484344
Epoch: 709 loss: 0.1725207269191742
Epoch: 710

Epoch: 907 loss: 0.16485202312469482
Epoch: 908 loss: 0.1654527634382248
Epoch: 909 loss: 0.16561754047870636
Epoch: 910 loss: 0.16610418260097504
Epoch: 911 loss: 0.16683292388916016
Epoch: 912 loss: 0.16712605953216553
Epoch: 913 loss: 0.167448490858078
Epoch: 914 loss: 0.16658832132816315
Epoch: 915 loss: 0.16578525304794312
Epoch: 916 loss: 0.1651129573583603
Epoch: 917 loss: 0.1652803272008896
Epoch: 918 loss: 0.165296733379364
Epoch: 919 loss: 0.16532555222511292
Epoch: 920 loss: 0.16546295583248138
Epoch: 921 loss: 0.1651187241077423
Epoch: 922 loss: 0.1651252806186676
Epoch: 923 loss: 0.16459964215755463
Epoch: 924 loss: 0.1643681675195694
Epoch: 925 loss: 0.16380617022514343
Epoch: 926 loss: 0.16396838426589966
Epoch: 927 loss: 0.16410279273986816
Epoch: 928 loss: 0.16523419320583344
Epoch: 929 loss: 0.16600792109966278
Epoch: 930 loss: 0.16778065264225006
Epoch: 931 loss: 0.16792213916778564
Epoch: 932 loss: 0.16757366061210632
Epoch: 933 loss: 0.1654309183359146
Epoch: 934 l

...

Epoch: 3978 loss: 0.14837831258773804
Epoch: 3979 loss: 0.1480003148317337
Epoch: 3980 loss: 0.14809384942054749
Epoch: 3981 loss: 0.14785301685333252
Epoch: 3982 loss: 0.1480838656425476
Epoch: 3983 loss: 0.14812687039375305
Epoch: 3984 loss: 0.14855019748210907
Epoch: 3985 loss: 0.1486607939004898
Epoch: 3986 loss: 0.14892035722732544
Epoch: 3987 loss: 0.14872334897518158
Epoch: 3988 loss: 0.14862282574176788
Epoch: 3989 loss: 0.1481519490480423
Epoch: 3990 loss: 0.14796537160873413
Epoch: 3991 loss: 0.1475074291229248
Epoch: 3992 loss: 0.14747171103954315
Epoch: 3993 loss: 0.1471991091966629
Epoch: 3994 loss: 0.14728689193725586
Epoch: 3995 loss: 0.14714325964450836
Epoch: 3996 loss: 0.1472698152065277
Epoch: 3997 loss: 0.1469220668077469
Epoch: 3998 loss: 0.14711613953113556
Epoch: 3999 loss: 0.14704440534114838
Epoch: 4000 loss: 0.14757131040096283
Epoch: 4001 loss: 0.14786501228809357
Epoch: 4002 loss: 0.14846277236938477
Epoch: 4003 loss: 0.14849038422107697
Epoch: 4004 loss: 0.

Epoch: 4198 loss: 0.14693258702754974
Epoch: 4199 loss: 0.14704962074756622
Epoch: 4200 loss: 0.14700782299041748
Epoch: 4201 loss: 0.14716069400310516
Epoch: 4202 loss: 0.14729538559913635
Epoch: 4203 loss: 0.14741301536560059
Epoch: 4204 loss: 0.14764782786369324
Epoch: 4205 loss: 0.14763034880161285
Epoch: 4206 loss: 0.14784546196460724
Epoch: 4207 loss: 0.1478327065706253
Epoch: 4208 loss: 0.14784298837184906
Epoch: 4209 loss: 0.1477610468864441
Epoch: 4210 loss: 0.1477302610874176
Epoch: 4211 loss: 0.14768609404563904
Epoch: 4212 loss: 0.1476895809173584
Epoch: 4213 loss: 0.14781107008457184
Epoch: 4214 loss: 0.14811314642429352
Epoch: 4215 loss: 0.148155078291893
Epoch: 4216 loss: 0.14824654161930084
Epoch: 4217 loss: 0.1478213518857956
Epoch: 4218 loss: 0.14754773676395416
Epoch: 4219 loss: 0.147119402885437
Epoch: 4220 loss: 0.1469399482011795
Epoch: 4221 loss: 0.1468113213777542
Epoch: 4222 loss: 0.14691771566867828
Epoch: 4223 loss: 0.1470608413219452
Epoch: 4224 loss: 0.1471

Epoch: 4417 loss: 0.14822432398796082
Epoch: 4418 loss: 0.14784975349903107
Epoch: 4419 loss: 0.14798444509506226
Epoch: 4420 loss: 0.1477324664592743
Epoch: 4421 loss: 0.1479925513267517
Epoch: 4422 loss: 0.14766080677509308
Epoch: 4423 loss: 0.14767013490200043
Epoch: 4424 loss: 0.14751511812210083
Epoch: 4425 loss: 0.147303968667984
Epoch: 4426 loss: 0.14706799387931824
Epoch: 4427 loss: 0.14702506363391876
Epoch: 4428 loss: 0.1468190997838974
Epoch: 4429 loss: 0.14672617614269257
Epoch: 4430 loss: 0.1465892195701599
Epoch: 4431 loss: 0.14643079042434692
Epoch: 4432 loss: 0.14642952382564545
Epoch: 4433 loss: 0.14641639590263367
Epoch: 4434 loss: 0.14669057726860046
Epoch: 4435 loss: 0.146794393658638
Epoch: 4436 loss: 0.14733457565307617
Epoch: 4437 loss: 0.1473744809627533
Epoch: 4438 loss: 0.1479337215423584
Epoch: 4439 loss: 0.14773495495319366
Epoch: 4440 loss: 0.14793285727500916
Epoch: 4441 loss: 0.14757660031318665
Epoch: 4442 loss: 0.14769065380096436
Epoch: 4443 loss: 0.14

Epoch: 4636 loss: 0.14653776586055756
Epoch: 4637 loss: 0.14660927653312683
Epoch: 4638 loss: 0.1471220701932907
Epoch: 4639 loss: 0.1468709409236908
Epoch: 4640 loss: 0.14721469581127167
Epoch: 4641 loss: 0.14695249497890472
Epoch: 4642 loss: 0.14712870121002197
Epoch: 4643 loss: 0.14695100486278534
Epoch: 4644 loss: 0.1471998691558838
Epoch: 4645 loss: 0.146919384598732
Epoch: 4646 loss: 0.14718268811702728
Epoch: 4647 loss: 0.14703209698200226
Epoch: 4648 loss: 0.14742127060890198
Epoch: 4649 loss: 0.14736782014369965
Epoch: 4650 loss: 0.14765521883964539
Epoch: 4651 loss: 0.14698459208011627
Epoch: 4652 loss: 0.1470097452402115
Epoch: 4653 loss: 0.1464497298002243
Epoch: 4654 loss: 0.14657403528690338
Epoch: 4655 loss: 0.1463080495595932
Epoch: 4656 loss: 0.1465677171945572
Epoch: 4657 loss: 0.1464746743440628
Epoch: 4658 loss: 0.1466725766658783
Epoch: 4659 loss: 0.14672395586967468
Epoch: 4660 loss: 0.14713792502880096
Epoch: 4661 loss: 0.1473960429430008
Epoch: 4662 loss: 0.1479

Epoch: 4856 loss: 0.1467324048280716
Epoch: 4857 loss: 0.1463553011417389
Epoch: 4858 loss: 0.14618977904319763
Epoch: 4859 loss: 0.14598841965198517
Epoch: 4860 loss: 0.14593406021595
Epoch: 4861 loss: 0.14598485827445984
Epoch: 4862 loss: 0.14614549279212952
Epoch: 4863 loss: 0.1463416963815689
Epoch: 4864 loss: 0.14664098620414734
Epoch: 4865 loss: 0.14655302464962006
Epoch: 4866 loss: 0.146792933344841
Epoch: 4867 loss: 0.14661000669002533
Epoch: 4868 loss: 0.14677540957927704
Epoch: 4869 loss: 0.14669495820999146
Epoch: 4870 loss: 0.14691229164600372
Epoch: 4871 loss: 0.147100567817688
Epoch: 4872 loss: 0.1471436768770218
Epoch: 4873 loss: 0.14705878496170044
Epoch: 4874 loss: 0.14692753553390503
Epoch: 4875 loss: 0.14709700644016266
Epoch: 4876 loss: 0.14720384776592255
Epoch: 4877 loss: 0.14704246819019318
Epoch: 4878 loss: 0.14684343338012695
Epoch: 4879 loss: 0.1466519981622696
Epoch: 4880 loss: 0.1463598757982254
Epoch: 4881 loss: 0.14641264081001282
Epoch: 4882 loss: 0.14635

In [7]:
tf.reset_default_graph()
model_path = "models/" + model
render = True


def prepareInput(obs, preObs, size):
    obs =obs.reshape([1,size])
    preObs = preObs.reshape([1,size])
    model_in = np.concatenate( (obs,obs-preObs), axis = 1 )
    model_in = normalize(model_in,mean = x_mean, std = x_std)
    model_in = model_in[x_mask.reshape([1,size*2])]
    model_in = np.expand_dims(model_in, axis=0)
    return model_in

def prepareOutput(modelOut):
    return unnormalize(modelOut,y_mean,y_std)

with tf.Session() as sess:
    # load the model
    saver = tf.train.import_meta_graph(model_path + '.ckpt.meta')
    saver.restore(sess,model_path+".ckpt")
    graph = tf.get_default_graph()
    policy_out = graph.get_tensor_by_name('pred/Tanh:0')#I_want_to_train_only_these/fc8/BiasAdd:0
    x_place = graph.get_tensor_by_name('input:0')
    
    def run_policy(policy_in):
        return sess.run(policy_out, feed_dict = {x_place: policy_in})
    
    env = gym.make(model)
    
    obs_len = env.observation_space.shape[0]
    obsPre = np.zeros((1,obs_len),dtype=np.float16)
    
    max_steps = 1000
    num_rollouts = 20
    returns = []
    observations = []
    actions = []
    
    for i in range(num_rollouts):
        obs = env.reset()
        observations.append(obs)
        done = False
        totalr = 0
        steps = 0
        
        model_in = prepareInput(obs,obsPre,obs_len)
        obsPre = obs
        preAction = run_policy(model_in)
        action = prepareOutput(preAction)
        obs, r, done, _ = env.step(action)
        while not done:
            model_in = prepareInput(obs,obsPre,obs_len)
            obsPre = obs
            preAction = run_policy(model_in)
            action = prepareOutput(preAction)
            obs, r, done, _ = env.step(action)
            totalr += r
            steps += 1
            if render:
                env.render()
            if steps % 100 == 0: print("%i/%i"%(steps, max_steps))
            if steps >= max_steps:
                break
        returns.append(totalr)
    
    print('returns', returns)
    print('mean return', np.mean(returns))
    print('std of return', np.std(returns))

INFO:tensorflow:Restoring parameters from models/Humanoid-v2.ckpt
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/yousof/Software/programming/anaconda3/envs/RL/lib/python3.5/site-packages/ipykernel_launcher.py:43: RuntimeWarning: divide by zero encountered in true_divide
/home/yousof/Software/programming/anaconda3/envs/RL/lib/python3.5/site-packages/ipykernel_launcher.py:43: RuntimeWarning: invalid value encountered in true_divide


Creating window glfw
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
100/1000
200/1000
300/1000
400/1000
500/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
100/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
100/1000
200/1000
300/1000
400/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
900/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/1000
700/1000
800/1000
100/1000
200/1000
300/1000
400/1000
500/1000
600/100